In [1]:
import json
import pandas as pd

In [2]:
def format_number(x):
    if isinstance(x, float):
        if x.is_integer():
            return int(x)  # Convert to integer if the float is a whole number
        else:
            return '{:.2f}'.format(x).rstrip('0').rstrip('.')  # Keep up to 2 decimals, remove trailing zeros
    return x

def flatten_json(json_data):
    data = []
    for circuit, circuit_value in json_data.items(): # circuit size
        for benchmark, benchmark_value in circuit_value.items(): # benchmark_name
            for layout, metrics in benchmark_value.items():
                if layout == 'init':
                    continue
                if 'lookahead_swap' not in metrics:
                    continue
                basic_swap = metrics.get('basic_swap', None)
                sabre_swap = metrics.get('sabre_swap', None)
                lookahead_swap = metrics.get('lookahead_swap', None)
                swap_change = ((lookahead_swap - sabre_swap) / sabre_swap * 100) if sabre_swap != 0 and sabre_swap is not None and lookahead_swap is not None else None
                
                basic_depth = metrics.get('basic_depth', None)
                sabre_depth = metrics.get('sabre_depth', None)
                lookahead_depth = metrics.get('lookahead_depth', None)
                depth_change = ((lookahead_depth - sabre_depth) / sabre_depth * 100) if sabre_depth != 0 and sabre_depth is not None and lookahead_depth is not None else None

                data.append({
                    "benchmark": benchmark,
                    "size": circuit, # circuit size
                    "gate": benchmark_value['init']['size'], # total gates
                    "depth": benchmark_value['init']['depth'], # circuit depth
                    "layout": layout,
                    "swap basic": basic_swap, # basic swap
                    "swap sabre": sabre_swap, # sabre swap
                    "swap lookahead": lookahead_swap, # lookahead swap
                    "swap percentage": swap_change,
                    "depth basic": basic_depth, # basic depth
                    "depth sabre": sabre_depth, # sabre depth
                    "depth lookahead": lookahead_depth, # lookahead depth
                    "depth percentage": depth_change
                })
    return pd.DataFrame(data)


In [3]:
def rearrange_columns(df, first_col_name):
    """
    Reorder the columns of the DataFrame so that the specified column is first.
    
    Parameters:
    - df: The input DataFrame.
    - first_col_name: The name of the column to move to the first position.
    
    Returns:
    - A DataFrame with the specified column moved to the first position.
    """
    # Check if the specified column exists in the DataFrame
    if first_col_name not in df.columns:
        raise ValueError(f"Column '{first_col_name}' does not exist in the DataFrame.")
    
    # Create a list of columns with the specified column first
    columns = [first_col_name] + [col for col in df.columns if col != first_col_name]
    
    # Reorder the DataFrame
    df_reordered = df[columns]
    
    return df_reordered

In [4]:
def open_json_from_file():
    with open("result/benchmarking_2.json") as json_file:
        return json.load(json_file)

dict_benchmark = open_json_from_file()
print(len(dict_benchmark["5"].keys()))

15


In [5]:
a = dict_benchmark["5"]["ghz"]["full_10_2"]
a_sabre_swap = a.get("sabre_swap", None)
print(isinstance(a_sabre_swap, int))

True


In [6]:
df = flatten_json(dict_benchmark)
df = df.map(format_number)

def rename_columns(df):
    df.columns = [
        "benchmark",
        # "n",
        "g",
        "d",
        "layout",
        "s basic", # basic swap
        "s sabre", # sabre swap
        "s look", # lookahead swap
        "swap (%)",
        "d basic", # basic depth
        "d swap", # sabre depth
        "d look", # lookahead depth
        "d (%)",
    ]
    return df
# df = rename_columns(df)

print(df)

    benchmark size  gate  depth            layout  swap basic  swap sabre  \
0          dj    5    36     11         full_10_2           0           0   
1          dj    5    36     11          full_7_3           0           3   
2          dj    5    36     11         ring_10_2          36           3   
3          dj    5    36     11          ring_7_3          24           3   
4          dj    5    36     11          grid_9_3           9           3   
..        ...  ...   ...    ...               ...         ...         ...   
380    wstate   15   253    135  t_horizontal_5_4          63          21   
381    wstate   15   253    135    t_vertical_5_4         126          45   
382    wstate   15   253    135          ring_7_3           0          15   
383    wstate   15   253    135          full_7_3           0          12   
384    wstate   15   253    135          ring_5_4           0          48   

     swap lookahead swap percentage  depth basic  depth sabre  \
0         

In [7]:
def save_latex_to_file(latex, circuit_size, filename=''):
    with open(f"table/table_{filename}_{circuit_size}.tex", "w") as file:
        file.write(latex)

In [8]:
def build_by_benchmark(df, circuit_size):
    df_c = df[df['size'] == f"{circuit_size}"]
    # df_n_5 = df_n_5.replace('_', r'\_', regex=True)
    df_c = df_c.sort_values(by=['gate', 'benchmark'])
    df_c = df_c.drop(columns=['size'])
    df_c = rename_columns(df_c)
    latex = df_c.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    latex = latex.replace('_', r'\_').replace('%', r'\%')
    save_latex_to_file(latex, circuit_size, 'benchmark')
    return df_c

In [9]:
df_5 = build_by_benchmark(df, 5)
df_10 = build_by_benchmark(df, 10)
df_15 = build_by_benchmark(df, 15)

In [10]:
def build_by_layout(df, circuit_size):
    df_b = df[df['size'] == f"{circuit_size}"]
    df_b = df_b.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
    df_b = df_b.drop(columns=['size']) # remove circuit size
    df_b = rename_columns(df_b)
    df_b = rearrange_columns(df_b, 'layout')
    latex = df_b.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    latex = latex.replace('_', r'\_').replace('%', r'\%')
    save_latex_to_file(latex, circuit_size, 'layout')
    return df_b

In [11]:
df_5 = build_by_layout(df, 5)
df_10 = build_by_layout(df, 10)
df_15 = build_by_layout(df, 15)

/tmp/ipykernel_16169/2156170491.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_b = df_b.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_16169/2156170491.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_b = df_b.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_16169/2156170491.py:3: DeprecationWarning: 